In [1]:
import psycopg2
import csv

conn = psycopg2.connect(
    host="localhost",
    database="mimiciv",
    user="postgres",
    password="GetPostgres1401"
)

cur = conn.cursor()

In [2]:
cur.execute("SELECT text FROM mimiciv_note.discharge")

discharge_data = cur.fetchall()

cur.close()
conn.close()

In [5]:
print("Discharge Data's first row:")
print(discharge_data[0][0])
print(f'Number of records: {len(discharge_data)}')

Discharge Data's first row:
 
Name:  ___                     Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: MEDICINE
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___
 
Chief Complaint:
Worsening ABD distension and pain 
 
Major Surgical or Invasive Procedure:
Paracentesis

 
History of Present Illness:
___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, 
bioplar, PTSD, presented from OSH ED with worsening abd 
distension over past week.  
Pt reports self-discontinuing lasix and spirnolactone ___ weeks 
ago, because she feels like "they don't do anything" and that 
she "doesn't want to put more chemicals in her." She does not 
follow Na-restricted diets. In the past week, she notes that she 
has been having worsening abd distension and discomfort. She 
denies ___ edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, 
dysuria. She had food poisoning a week ago from eating stal

In [6]:
import re
import json

def parse_text_to_json(text):
    json_data = {}

    # Extracting Name and Unit No
    name_match = re.search(r'Name:\s+(.*?)\s+Unit No:\s+(.*?)\s+', text)
    if name_match:
        json_data['Name'] = name_match.group(1).strip()
        json_data['Unit No'] = name_match.group(2).strip()

    # Extracting Admission Date and Discharge Date
    date_match = re.search(r'Admission Date:\s+(.*?)\s+Discharge Date:\s+(.*?)\s+', text)
    if date_match:
        json_data['Admission Date'] = date_match.group(1).strip()
        json_data['Discharge Date'] = date_match.group(2).strip()

    # Extracting Date of Birth and Sex
    dob_match = re.search(r'Date of Birth:\s+(.*?)\s+Sex:\s+(.*?)\s+', text)
    if dob_match:
        json_data['Date of Birth'] = dob_match.group(1).strip()
        json_data['Sex'] = dob_match.group(2).strip()

    # Extracting Service
    service_match = re.search(r'Service:\s+(.*?)\s+\n', text)
    if service_match:
        json_data['Service'] = service_match.group(1).strip()

    # Extracting Allergies
    allergies_match = re.search(r'Allergies:\s+(.*?)\s+\n', text)
    if allergies_match:
        json_data['Allergies'] = allergies_match.group(1).strip()

    # Extracting Attending
    attending_match = re.search(r'Attending:\s+(.*?)\s+', text)
    if attending_match:
        json_data['Attending'] = attending_match.group(1).strip()

    # Extracting Chief Complaint
    chief_complaint_match = re.search(r'Chief Complaint:\s+(.*?)\s+\n', text)
    if chief_complaint_match:
        json_data['Chief Complaint'] = chief_complaint_match.group(1).strip()

    # Extracting Major Surgical or Invasive Procedure
    procedure_match = re.search(r'Major Surgical or Invasive Procedure:\s+(.*?)\s+\n', text)
    if procedure_match:
        json_data['Major Surgical or Invasive Procedure'] = procedure_match.group(1).strip()

    # Extracting History of Present Illness
    present_illness_match = re.search(r'History of Present Illness:\s+(.*?)\n\n', text, re.DOTALL)
    if present_illness_match:
        if not present_illness_match.group(1).strip().startswith("___\n"): 
            json_data['History of Present Illness'] = present_illness_match.group(1).strip()
        else:
            json_data['History of Present Illness'] = "___"

    # Extracting Past Medical History
    past_illness_match = re.search(r'Past Medical History:\s+(.*?)\n\n', text, re.DOTALL)
    if past_illness_match:
        if not past_illness_match.group(1).strip().startswith("___\n"): 
            json_data['Past Medical History'] = past_illness_match.group(1).strip()
        else:
            json_data['Past Medical History'] = "___"

    # Extracting Social History
    social_match = re.search(r'Social History:\s+(.*?)\n\n', text, re.DOTALL)
    if social_match:
        if not social_match.group(1).strip().startswith("___\n"): 
            json_data['Social History'] = social_match.group(1).strip()
        else:
            json_data['Social History'] = "___"

    # Extracting Family History
    family_match = re.search(r'Family History:\s+(.*?)\n\n', text, re.DOTALL)
    if family_match:
        if not family_match.group(1).strip().startswith("___\n"): 
            json_data['Family History'] = family_match.group(1).strip()
        else:
            json_data['Family History'] = "___"

    # Extracting Brief Hospital Course
    brief_hospital_course = re.search(r'Brief Hospital Course:\s+(.*?)\s+\nMedications on Admission:', text, re.DOTALL)
    if brief_hospital_course:
        if not brief_hospital_course.group(1).strip().startswith("___\n"): 
            json_data['Brief Hospital Course'] = brief_hospital_course.group(1).strip()
        else:
            json_data['Brief Hospital Course'] = "___"
    
    # Extracting Discharge Medications
    discharge_medications = re.search(r'Discharge Medications:\s+(.*?)\n\n', text, re.DOTALL)
    if discharge_medications:
        if not discharge_medications.group(1).strip().startswith("___\n"): 
            json_data['Discharge Medications'] = discharge_medications.group(1).strip()
        else:
            json_data['Discharge Medications'] = "___"

    # Extracting Discharge Disposition
    discharge_disposition = re.search(r'Discharge Disposition:\s+(.*?)\n \n', text, re.DOTALL)
    if discharge_disposition:
        if not discharge_disposition.group(1).strip().startswith("___\n"): 
            json_data['Discharge Disposition'] = discharge_disposition.group(1).strip()
        else:
            json_data['Discharge Disposition'] = "___"

    # Extracting Discharge Diagnosis
    discharge_diagnosis = re.search(r'Discharge Diagnosis:\s+(.*?)\n\n', text, re.DOTALL)
    if discharge_diagnosis:
        if not discharge_diagnosis.group(1).strip().startswith("___\n"): 
            json_data['Discharge Diagnosis'] = discharge_diagnosis.group(1).strip()
        else:
            json_data['Discharge Diagnosis'] = "___"

    # Extracting Discharge Condition
    discharge_condition = re.search(r'Discharge Condition:\s+(.*?)\n\n', text, re.DOTALL)
    if discharge_condition:
        if not discharge_condition.group(1).strip().startswith("___\n"): 
            json_data['Discharge Condition'] = discharge_condition.group(1).strip()
        else:
            json_data['Discharge Condition'] = "___"

    # Extracting Discharge Instructions
    discharge_Instructions = re.search(r'Discharge Instructions:\s+(.*?)\n \n', text, re.DOTALL)
    if discharge_Instructions:
        if not discharge_Instructions.group(1).strip().startswith("___\n"): 
            json_data['Discharge Instructions'] = discharge_Instructions.group(1).strip()
        else:
            json_data['Discharge Instructions'] = "___"

    # Extracting Followup Instructions
    followup_Instructions = re.search(r'Followup Instructions:\s+(.*?)\n', text, re.DOTALL)
    if followup_Instructions:
        if not followup_Instructions.group(1).strip().startswith("___\n"): 
            json_data['Followup Instructions'] = followup_Instructions.group(1).strip()
        else:
            json_data['Followup Instructions'] = "___"

            

            

    return json.dumps(json_data, indent=4)


In [7]:
# Example usage
text = discharge_data[0][0]
json_data = parse_text_to_json(text)
print(json_data)

print(discharge_data[0][0])

{
    "Name": "___",
    "Unit No": "___",
    "Admission Date": "___",
    "Discharge Date": "___",
    "Date of Birth": "___",
    "Sex": "F",
    "Service": "MEDICINE",
    "Allergies": "No Known Allergies / Adverse Drug Reactions",
    "Attending": "___",
    "Chief Complaint": "Worsening ABD distension and pain",
    "Major Surgical or Invasive Procedure": "Paracentesis",
    "History of Present Illness": "___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, \nbioplar, PTSD, presented from OSH ED with worsening abd \ndistension over past week.  \nPt reports self-discontinuing lasix and spirnolactone ___ weeks \nago, because she feels like \"they don't do anything\" and that \nshe \"doesn't want to put more chemicals in her.\" She does not \nfollow Na-restricted diets. In the past week, she notes that she \nhas been having worsening abd distension and discomfort. She \ndenies ___ edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, \ndysuria. She had food poisoning a week ag

In [8]:
json_data = list()
for row in discharge_data:
    json_data.append(parse_text_to_json(row[0]))

In [9]:
counter_map = {"Name": 331794,
"Unit No": 331794,
"Admission Date": 331794,
"Discharge Date": 331794,
"Date of Birth": 331794,     
"Sex": 331794,  
"Service": 331794,  
"Allergies": 331794,  
"Attending": 331794,  
"Chief Complaint": 331794,  
"Major Surgical or Invasive Procedure": 331794,  
"History of Present Illness": 331794,  
"Past Medical History": 331794,  
"Social History": 331794,  
"Family History": 331794,  
"Brief Hospital Course": 331794,  
"Discharge Medications": 331794,  
"Discharge Disposition": 331794,  
"Discharge Diagnosis": 331794,  
"Discharge Condition": 331794,  
"Discharge Instructions": 331794,  
"Followup Instructions": 331794}

maps = []
for data in json_data:
    obj = json.loads(data)

    my_map = {}
    for key, value in obj.items():
        my_map[key] = value
    maps.append(my_map)

    for key in counter_map.keys():
        if (key in my_map.keys()):
            if (my_map[key] != "___"):
                pass
            else:
                counter_map[key] -= 1
        else:
            counter_map[key] -= 1

print(counter_map)
    

{'Name': 15750, 'Unit No': 0, 'Admission Date': 19, 'Discharge Date': 33, 'Date of Birth': 10, 'Sex': 331777, 'Service': 331039, 'Allergies': 326486, 'Attending': 257348, 'Chief Complaint': 311727, 'Major Surgical or Invasive Procedure': 288747, 'History of Present Illness': 325433, 'Past Medical History': 325376, 'Social History': 9110, 'Family History': 319734, 'Brief Hospital Course': 275343, 'Discharge Medications': 317952, 'Discharge Disposition': 327444, 'Discharge Diagnosis': 305695, 'Discharge Condition': 315794, 'Discharge Instructions': 328280, 'Followup Instructions': 0}


In [11]:
!pip install plotly

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import plotly.graph_objects as go

# Convert counter_map to lists for plotting
keys = list(counter_map.keys())
values = list(counter_map.values())

# Create bar chart
fig = go.Figure(data=[go.Bar(x=keys, y=values)])
fig.update_layout(title='Number of non-empty fields in dataset',
                  xaxis_title='Keys', yaxis_title='Values')

fig.show()


In [18]:
# Test the map
paragraphs = maps[2]["Family History"].split('\n \n')
Family_History = paragraphs[0]
Family_History = Family_History.replace('\n', '').strip()
print(Family_History)

She a total of five siblings, but she is not  talking to most of them. She only has one brother that she is in  touch with and lives in ___. She is not aware of any known GI or liver disease in her family.


In [19]:
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk import pos_tag
from nltk import ne_chunk
from transformers import AutoTokenizer, AutoModel
import spacy
import pandas as pd
from spacy import displacy
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from nltk import sent_tokenize

In [20]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

nltk_stopwords = stopwords.words('english')
stemmer = PorterStemmer()

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
from transformers import pipeline

nlp = pipeline("sentiment-analysis")

family_histories = set()

for map in maps:
    family_history = map.get("Family History", "") 
    if family_history.strip():  
        paragraphs = family_history.split('\n \n')
        family_history = paragraphs[0]
        family_history = family_history.replace('\n', '').strip()
        if family_history != '___':
            family_histories.add(family_history)

positive_count = 0
negative_count = 0
sentiments = []

print(len(family_histories))
for i, family_history in enumerate(family_histories):
    if i % 10000 == 0:
        print(i)
    try:
        result = nlp(family_history)
        
        sentiments.extend(result)
        for item in result:
            label = item['label']
            if label == 'NEGATIVE':
                negative_count += 1
            elif label == 'POSITIVE':
                positive_count += 1
            else:
                print(label)

    except Exception as e:
        continue

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

138293
0


Token indices sequence length is longer than the specified maximum sequence length for this model (830 > 512). Running this sequence through the model will result in indexing errors


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000


In [23]:
import plotly.express as px

labels = ['POSITIVE', 'NEGATIVE']
sizes = [positive_count, negative_count]

fig = px.pie(values=sizes, names=labels, 
             color=labels,
             color_discrete_map={'POSITIVE': 'green', 'NEGATIVE': 'red'},
             title='Sentiment Analysis of Family Histories')

fig.show()


In [25]:
nlp = spacy.load("en_core_web_sm")

In [63]:
!pip install TextBlob

  Using cached textblob-0.17.1-py2.py3-none-any.whl (636 kB)


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [119]:
from textblob import TextBlob

positive_count = 0
negative_count = 0
neutral_count = 0
sentiments = []

print(len(family_histories))
for i, family_history in enumerate(family_histories):
    if i % 10000 == 0:
        print(i)
    try:
        blob = TextBlob(family_history)
        if blob.sentiment.polarity < 0:
            negative_count += 1
        elif blob.sentiment.polarity > 0:
            positive_count += 1
        else:
            neutral_count += 1

        sentiments.extend(blob)
        

    except Exception as e:
        continue

138293
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000


In [123]:
labels = ['POSITIVE', 'NEGATIVE', 'NEUTRAL']
sizes = [positive_count, negative_count, neutral_count]

fig = px.pie(values=sizes, names=labels, 
             color=labels,
             color_discrete_map={'POSITIVE': 'green', 'NEGATIVE': 'red', 'NEUTRAL': 'blue'},
             title='Sentiment Analysis of Family Histories')

fig.show()


In [77]:

from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

model_name = "emilyalsentzer/Bio_ClinicalBERT"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

In [94]:
def classify_sentence(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs[0], dim=-1)
    prediction = torch.argmax(probabilities).item()
    return prediction


In [117]:
def check_substance_abuse(sentences):
    addiction = 0
    no_addiction = 0
    for sentence in tqdm(sentences, desc="Checking Substance Abuse"):
        try:
            prediction = classify_sentence(sentence)
            if prediction == 0:
                addiction += 1
                #print("Prediction: Contains signs of addiction or abuse")
            else:
                no_addiction += 1
                #print("Prediction: Does not contain signs of addiction or abuse")
        except Exception as e:
            continue
    return addiction, no_addiction

In [112]:
from tqdm import tqdm

substances = ["alcohol", "tobacco", "cannabis", "weed", "marijuana", "drug"]

substance_sentences = []

for i, note in enumerate(tqdm(discharge_data[:10000], desc="Processing notes", leave=True)):
    note = note[0].replace('\n', '').strip()
    sentences = sent_tokenize(note)
    note_sentences = ""
    for sentence in sentences:
        if any(substance in sentence.lower() for substance in substances):
            note_sentences += sentence
    if note_sentences != "":
        substance_sentences.append(note_sentences)

Processing notes:   0%|          | 0/10000 [00:00<?, ?it/s]

Processing notes: 100%|██████████| 10000/10000 [00:28<00:00, 352.58it/s]


In [118]:
addiction, no_addiction = check_substance_abuse(substance_sentences)

Checking Substance Abuse: 100%|██████████| 6107/6107 [26:09<00:00,  3.89it/s]    


In [121]:
sizes = [addiction, no_addiction, 10000 - addiction - no_addiction]

# Labels for the subsets
labels = ['Addiction', 'No-addiction', 'Other']

# Create an interactive pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=sizes)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=['gold', 'yellowgreen', 'lightcoral'], line=dict(color='#000000', width=2)))
fig.update_layout(title_text="Distribution of Subsets within the Larger Set")
fig.show()